# Quickstart Guide for Strands Agents

Strands Agents is a powerful framework for building AI agents that can interact with AWS services and perform complex tasks. This quickstart guide will help you get started with creating your first Strands agent.

## Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Basic understanding of Python programming

Lets get started !

In [ ]:
# Install Strands using pip

# !uv add strands-agents strands-agents-tools 

!uv sync

Resolved 226 packages in 2ms
Audited 221 packages in 0.13ms



## Creating Your First Agent

Lets get an overview of the agentic components needed.

### 1. Create a simple Agent:

This will create an agent with the default model provider, [Amazon Bedrock](https://aws.amazon.com/bedrock/), and the default model, Claude 3.7 Sonnet, in the region of your AWS setup. While the agent runs in the same local environment as it is being invoked, Amazon Bedrock models will run in an AWS account and your agent will invoke the model in the cloud account. The architecture looks as following:

<div style="text-align:center">
    <img src="images/simple_agent.png" width="75%" />
</div>

In [7]:
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow") 
from strands import Agent

# Initialize your agent
agent = Agent(
    model="apac.anthropic.claude-3-5-sonnet-20240620-v1:0",  # Optional: Specify the model ID
    system_prompt="You are a helpful assistant that provides concise responses."
)

# Send a message to the agent
response = agent("Hello! Tell me a joke.")

Sure, here's a short joke for you:

Why don't scientists trust atoms?
Because they make up everything!

### 2. Add Tools to the Agent:

The [strands-agents-tools](https://github.com/strands-agents/tools) repository provides some in-built tools which you can import. You can also create custom tools using the `@tool` decorator. We can create agents with built-in and custom tools. For instance, adding the built-in tool of a calculator and a custom tool for getting the weather you get the following architecture:
<div style="text-align:center">
    <img src="images/agent_with_tools.png" width="75%" />
</div>

Implementing this architecture you have the following:

In [8]:
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

agent = Agent(
    model="apac.anthropic.claude-3-5-sonnet-20240620-v1:0",  # Optional: Specify the model ID
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.")

response = agent("What is the weather today")
print(response)

To provide you with the current weather information, I'll need to use the weather tool. Let me fetch that data for you.
Tool #1: weather


Based on the information I've received, the weather today is sunny. It's a great day to spend some time outdoors if you have the opportunity! Is there anything specific you'd like to know about today's weather, or do you have any plans you're considering based on this sunny forecast?

Based on the information I've received, the weather today is sunny. It's a great day to spend some time outdoors if you have the opportunity! Is there anything specific you'd like to know about today's weather, or do you have any plans you're considering based on this sunny forecast?



### Invoking tool directly

For some applications it is important to directly call the tool. For instance, you might want to debug the tool, pre-populate the agent knowledge with your customer's information or using a tool inside of another tool. In Strands you can do it using the ``tool`` method of your agent followed by the tool name

In [9]:
# Alternatively, you can invoke the tool directly like so:
agent.tool.calculator(expression="sin(x)", mode="derive", wrt="x", order=2)

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭─────────────────┬───────────────────────────╮                                                                │
│  │ Operation       │ Calculate 2-th Derivative │                                                                │
│  │ Input           │ sin(x)                    │                                                                │
│  │ Result          │ -sin(x)                   │                                                                │
│  │ With respect to │ x                         │                                                                │
│  ╰─────────────────┴───────────────────────────╯                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'status': 'success',
 'content': [{'text': 'Result: -sin(x)'}],
 'toolUseId': 'tooluse_calculator_209144671'}


### 3. Changing the log level and format:

Strands SDK uses Python's standard `logging` module to provide visibility into its operations.

The Strands Agents SDK implements a straightforward logging approach:

1. **Module-level Loggers**: Each module in the SDK creates its own logger using logging.getLogger(__name__), following Python best practices for hierarchical logging.
2. **Root Logger**: All loggers in the SDK are children of the "strands" root logger, making it easy to configure logging for the entire SDK.
3. **Default Behavior**: By default, the SDK doesn't configure any handlers or log levels, allowing you to integrate it with your application's logging configuration.

To enable logging for the Strands Agents SDK, you can configure the **"strands"** logger. If you want to change the log level, for example during debugging, or modify the log format, you can set the logger configuration as follows:

In [10]:
import logging
from strands import Agent

# Enables Strands debug log level
logging.getLogger("strands").setLevel(logging.DEBUG) # or logging.INFO

# Sets the logging format and streams logs to stderr
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

agent = Agent(model="apac.anthropic.claude-3-5-sonnet-20240620-v1:0")  # Optional: Specify the model ID
agent("Hello!")

DEBUG | strands.models.bedrock | config=<{'model_id': 'apac.anthropic.claude-3-5-sonnet-20240620-v1:0'}> | initializing
DEBUG | strands.models.bedrock | region=<ap-southeast-2> | bedrock client created
DEBUG | strands.tools.registry | tools_dir=<e:\OneDrive\OneDriveOnitbuddy\OneDrive\workbench\exampletest\agenticAIeducation\5_strand\01-first-agent\tools> | tools directory not found
DEBUG | strands.tools.registry | tool_modules=<[]> | discovered
DEBUG | strands.tools.registry | tool_count=<0>, success_count=<0> | finished loading tools
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_count=<0> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_count=<0> | tools configured
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x000001DD55105450>> | streaming messages
DEBUG | strands.models.bedrock | formatting request
DEBUG | strands.

Hello! It's nice to meet you. How can I assist you today? Feel free to ask me any questions or let me know if there's a topic you'

DEBUG | strands.models.bedrock | finished streaming response from model
DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | message_count=<2>, window_size=<40> | skipping context reduction


d like to discuss.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Hello! It's nice to meet you. How can I assist you today? Feel free to ask me any questions or let me know if there's a topic you'd like to discuss."}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[22.973682641983032], traces=[<strands.telemetry.metrics.Trace object at 0x000001DD558CD250>], accumulated_usage={'inputTokens': 9, 'outputTokens': 41, 'totalTokens': 50}, accumulated_metrics={'latencyMs': 1600}), state={})


### 4. Model Provider

The default model provider is [Amazon Bedrock](https://aws.amazon.com/bedrock/) and the default model is Claude 3.7 Sonnet in the region of your current AWS environment

You can specify a different model in Amazon Bedrock providing the model ID string directly:

In [11]:
from strands import Agent

agent = Agent(model="anthropic.claude-3-5-haiku-20241022-v1:0")
print(agent.model.config)

DEBUG | strands.models.bedrock | config=<{'model_id': 'anthropic.claude-3-5-haiku-20241022-v1:0'}> | initializing
DEBUG | strands.models.bedrock | region=<ap-southeast-2> | bedrock client created
DEBUG | strands.tools.registry | tools_dir=<e:\OneDrive\OneDriveOnitbuddy\OneDrive\workbench\exampletest\agenticAIeducation\5_strand\01-first-agent\tools> | tools directory not found
DEBUG | strands.tools.registry | tool_modules=<[]> | discovered
DEBUG | strands.tools.registry | tool_count=<0>, success_count=<0> | finished loading tools


{'model_id': 'anthropic.claude-3-5-haiku-20241022-v1:0'}



For more control over the model configuration, you can create a `BedrockModel` provider instance:

In [12]:
import boto3
from strands import Agent
from strands.models import BedrockModel

# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="anthropic.claude-3-5-haiku-20241022-v1:0",
    region_name='ap-southeast-2',
    temperature=0.3,
)

agent = Agent(model=bedrock_model)

DEBUG | strands.models.bedrock | config=<{'model_id': 'anthropic.claude-3-5-haiku-20241022-v1:0', 'temperature': 0.3}> | initializing
DEBUG | strands.models.bedrock | region=<ap-southeast-2> | bedrock client created
DEBUG | strands.tools.registry | tools_dir=<e:\OneDrive\OneDriveOnitbuddy\OneDrive\workbench\exampletest\agenticAIeducation\5_strand\01-first-agent\tools> | tools directory not found
DEBUG | strands.tools.registry | tool_modules=<[]> | discovered
DEBUG | strands.tools.registry | tool_count=<0>, success_count=<0> | finished loading tools



More details on the available model providers on the [Model Provider Quickstart page](https://strandsagents.com/0.1.x/user-guide/quickstart/#model-providers)


**Congratulations !! Now you have learned how to build a simple agent using Strands!!**

## [Optional] Lets Build a Task-Specific Agent - RecipeBot 🍽️

Lets create a practical example of a task-specific agent. We create a `RecipeBot` that recommends recipes and answers any cooking related questions. Lets dive in !!

Here's what we will create :

<div style="text-align:center">
    <img src="images/interactive_recipe_agent.png" width="75%" />
</div>

In [14]:
# Install the required packages # Also install strands-agents strands-agents-tools if you haven't already
!uv add ddgs 

Resolved 242 packages in 784ms
Prepared 2 packages in 914ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 2 packages in 39ms
 + ddgs==9.5.2
 + primp==0.15.0


In [15]:
from strands import Agent, tool
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException
import logging

# Configure logging
logging.getLogger("strands").setLevel(logging.INFO)

# Define a websearch tool
@tool
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"


In [16]:
# Create a recipe assistant agent
recipe_agent = Agent(
    model="apac.anthropic.claude-3-5-sonnet-20240620-v1:0",  # Optional: Specify the model ID
    system_prompt="""You are RecipeBot, a helpful cooking assistant.
    Help users find recipes based on ingredients and answer cooking questions.
    Use the websearch tool to find recipes when users mention ingredients or to
    look up cooking information.""",
    # Import the websearch tool we created above
    tools=[websearch],
)

In [17]:
response = recipe_agent("Suggest a recipe with chicken and broccoli.")

# Other examples:
# response = recipe_agent("How do I cook quinoa?")
# response = recipe_agent("How can I substitute white wine in shrimp pasta?")
# response = recipe_agent("What are the health benefits of asparagus?")
print(response)

Certainly! I'd be happy to suggest a recipe with chicken and broccoli. To get the most up-to-date and relevant recipe suggestions, I'll use the websearch tool to find some options for you.
Tool #1: websearch


Great! I've found some delicious and easy chicken and broccoli recipes for you. Here's a suggestion based on the search results:

Chicken and Broccoli Stir Fry

This recipe is quick, easy, and perfect for a weeknight dinner. Here's a summary of the recipe:

Ingredients:
- Chicken breast, cut into bite-sized pieces
- Fresh broccoli florets
- Garlic
- Ginger
- Soy sauce
- Brown sugar
- Vegetable oil for cooking

Instructions:
1. Heat oil in a large skillet or wok over medium-high heat.
2. Add chicken pieces and cook until they're golden brown and almost cooked through.
3. Add broccoli florets to the pan and stir-fry for a few minutes until they're bright green and slightly tender.
4. In a small bowl, mix soy sauce, minced garlic, grated ginger, and a bit of brown sugar to create th

#### And that's it! We now have a running usecase agent with tools in just a few lines of code 🥳.

For more detailed quickstart guide, check out the [Strands documentation](https://strandsagents.com/0.1.x/user-guide/quickstart/).

### [Optional] Run RecipeBot via CLI: 
you can run the agent in interactive mode via the command line (for instance using the terminal on SageMaker Studio) through the python script provided in `02_simple_interactive_usecase/recipe_bot.py`. This allows you to interact with the agent in a more dynamic way, sending messages and receiving responses via the CLI.
Run these commands on a command line interface to run the agent in interactive mode:

```cli
cd samples/01-tutorials/01-fundamentals/01-first-agent/02-simple-interactive-usecase/
pip install -r requirements.txt
python recipe_bot.py
```

With this, you can talk to the bot via a command line interface(CLI).